<img align="left" src="https://project.lsst.org/sites/default/files/Rubin-O-Logo_0.png" width=250 style="padding: 10px"> \n<br><b>Variability of Young Stellar Objects — Internship Report</b>  \n**Author:** Chiara Virzì  \n**Date:** April 2025\n

This report outlines the work I conducted during my internship under the supervision of Dr. Rosaria Bonito at the Osservatorio Astronomico di Palermo. The focus of the internship was the analysis of variability in Young Stellar Objects (YSOs), using both photometric and spectroscopic data, in preparation for the upcoming Rubin LSST survey.

# **1. Introduction**  \nThe aim of this project is to study spectral variability in Young Stellar Objects (YSOs), with a focus on the Hα emission line, across timescales of several months. As a Rubin LSST data rights holder, I developed methods to identify accretion signatures in archival spectra and correlated these features with photometric variability. The Hα line serves as a key diagnostic for accretion processes, mass-loss activity, and the evolutionary stages of pre-main-sequence stars.

# **2. Young Stellar Object Science**  \nYSOs are stars in their early formation stages, still undergoing contraction and typically surrounded by disks of gas and dust. These stars are known for their strong variability, driven by accretion, magnetic activity, and interactions with surrounding circumstellar material.

### **YSO Evolutionary Classes**  \nYSOs are classified based on their spectral energy distributions (SEDs) and infrared excess:\n- **Class 0**: Deeply embedded protostars, observed mainly in the submillimeter range.\n- **Class I**: Protostars with infalling envelopes, showing strong infrared excess and outflows.\n- **Class II**: Classical T Tauri Stars (CTTS) with developed accretion disks and active mass transfer.\n- **Class III**: Weak-lined T Tauri Stars (WTTS) with little or no disk and minimal accretion signatures.

Understanding Class II and III sources is crucial for studying late-stage star formation and early disk evolution—key elements for understanding planet formation. [4](#ref4)

## **3. Rubin LSST and 4MOST**  \nThe **Rubin Observatory’s LSST** (Legacy Survey of Space and Time) will image the sky in six bands (*u, g, r, i, z, y*) over ten years, enabling both short- and long-term variability studies.  \n<img align="center" src="https://github.com/Chiaravir/internship/blob/main/immaginireport/filters_lsst.jpeg" width=500><br>\n<em><small>Fig 1: LSST filter transmission curves, including atmospheric and telescope throughput.</small></em>

The **4MOST** facility will provide time-resolved spectroscopy for variable-source follow-up. In this study, the NGC 2264 region—also observed by Gaia-ESO—serves as a testbed for the Rubin–4MOST synergy.

### **Brokers**  \nReal-time brokers (ANTARES, ALeRCE) filter and classify survey alerts, providing light curves, forced photometry, and ML-based object classification to facilitate follow-up observations.